In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
from tqdm import tqdm

2022-03-31 18:58:28.146652: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def build_model(input_shape=[50], lr_rate=0.02, lamb=0.8, seed_num=1993, decay=True, steps=30):
    tf.random.set_seed(seed_num)
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    model.add(keras.layers.Dense(1, kernel_regularizer=keras.regularizers.l1(lamb), use_bias=False,
                                 kernel_initializer=keras.initializers.RandomNormal(mean=0, stddev=1)))
    if decay:
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr_rate,
                                                                  decay_steps=steps, decay_rate=0.95, staircase=True)

        optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
    else:
        optimizer = keras.optimizers.SGD(learning_rate=lr_rate)
    model.compile(loss="mse", optimizer=optimizer)

    return model

In [5]:
np.random.seed(2022)
n = 100; p = 200
xmat = np.random.normal(size = (n, p)).astype("float64")
beta = np.zeros(p)
beta[:int(0.05*p)] = 1.0
y = np.dot(xmat, beta) + np.random.normal(size = n)

lamb = np.sqrt(2 * np.log(p) / n) 

lr_rate = 0.01; seed = 2022; steps = 200

In [6]:
keras.backend.set_floatx("float64")

nn_lasso = keras.wrappers.scikit_learn.KerasRegressor(build_model,verbose = 0)
nn_lasso.set_params(input_shape = [xmat.shape[1]], lamb = lamb, lr_rate = lr_rate, 
                    steps = steps, decay = True, seed_num = seed)
            
t1 = time.time()
nn_lasso.fit(xmat, y, validation_data=(xmat, y), epochs=10000,
         callbacks=[keras.callbacks.EarlyStopping(patience=20)])
t2 = time.time()

2022-03-31 19:03:04.958664: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-03-31 19:03:05.013750: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 19:03:05.014206: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0d:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-03-31 19:03:05.014223: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-03-31 19:03:05.016515: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-03-31 19:03:05.016539: I tensorflow/stream_executor

In [7]:
t2-t1

3.810309886932373

In [24]:
beta = (nn_lasso.model.layers[0].get_weights()[0]).reshape(p,)

In [25]:
beta[:20]

array([ 0.6101553 ,  0.86606187,  0.67166085,  0.91051458,  0.61152677,
        0.68742471,  0.67288994,  0.89709712,  0.82466948,  0.78447595,
       -0.01059803, -0.28532738, -0.0181754 ,  0.01256181,  0.00351842,
        0.00093109, -0.08106986,  0.00299414, -0.00914166, -0.01019493])